# Haunted House Images Generation

In [ ]:
# Previously had later versions but downgraded for StyleGAN2 ADA.
# torch 1.12.1
# torchvision 0.13.1

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.4 kB/s 
     |████████████████████████████████| 17.4 MB 77.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.


In [ ]:
!nvidia-smi

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 120 kB 15.5 MB/s 


In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 12.61 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
!ls /content/gdrive/MyDrive/StyleGAN_Practice/images/haunted_houses

### Check if there are any unfit images 
The preprocessing of raw data using `process_img.py` only ensures the size of the images. It is important that all images have the same dimensions and color depth. This code can identify images that have issues.

In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

In [ ]:
IMAGE_PATH = '/content/gdrive/MyDrive/StyleGAN_Practice/images/haunted_houses/'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz

  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
import torch
import torchvision.transforms.functional as fn
from PIL import Image

In [ ]:
# img = Image.open("/content/gdrive/MyDrive/StyleGAN_Practice/images/haunted_houses/hauntedhousephotography1.jpg")
# print("This is size of original image:", img.size, "\n")

This is size of original image: (512, 512) 



In [ ]:
# image_list = os.listdir(IMAGE_PATH)
# image_list[120]

### Convert Images (from raw images to datasets)

In [ ]:
# To empty datasets folders in case of error or redo it. 

!rm -R /content/gdrive/MyDrive/StyleGAN_Practice/datasets/*

In [ ]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/gdrive/MyDrive/StyleGAN_Practice/images/haunted_houses --dest /content/gdrive/MyDrive/StyleGAN_Practice/datasets/haunted_houses


100% 150/150 [00:08<00:00, 18.63it/s]


### Perform Initial Training (1st time)

In [ ]:
# Modify these to suit your needs
EXPERIMENTS = "/content/gdrive/MyDrive/StyleGAN_Practice/experiments"
DATA = "/content/gdrive/MyDrive/StyleGAN_Practice/datasets/haunted_houses"
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

### Resume Training after Disconnections

In [ ]:
# Modify these to suit your needs
EXPERIMENTS = "/content/gdrive/MyDrive/StyleGAN_Practice/experiments"
NETWORK = "network-snapshot-000120.pkl"
RESUME = os.path.join(EXPERIMENTS, "00012-haunted_houses-auto1-resumecustom", NETWORK)
DATA = "/content/gdrive/MyDrive/StyleGAN_Practice/datasets/haunted_houses"
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

### Generating Images - Run Multiple Times using Different pkl Files

In [ ]:
root_folder = "/content/gdrive/MyDrive/StyleGAN_Practice/experiments/"
result_folder = "00014-haunted_houses-auto1-resumecustom"
network_to_load = "network-snapshot-000000.pkl"
network_pkl = os.path.join(root_folder, result_folder, network_to_load)
outputs = "/content/gdrive/MyDrive/StyleGAN_Practice/output/"
num_images = 20

In [ ]:
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/generate.py --network={network_pkl} --outdir={outputs} --trunc=1.0 --seeds=0-{num_images-1}"
!{cmd}
# !python stylegan2-ada/generate.py --network={network_pkl} --outdir=out --trunc=1.0 --seeds=0-{num_images-1}

### Outputs were reviewed and some better images were manually selected. Display the examples in a grid.

In [6]:
from PIL import Image
import os

file_path='/content/gdrive/My Drive/Portfolio-Full-Backup/StyleGAN2-haunted-houses/outputs-selected'
imgs = os.listdir(file_path)

images=[]
for img in imgs:
    f_img = Image.open(os.path.join(file_path, img))
    images.append(f_img)

In [7]:
# To disply images in grid

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


In [8]:
image_grid(images, 2, 5)

Output hidden; open in https://colab.research.google.com to view.